In [1]:
# Generative graphs
%matplotlib inline
import networkx as nx

from collections import defaultdict

from prepare_datasets import preprocess_generative_graph
from graphs import generate_graph, attach_graph_attributes
from measurements import get_graph_measurements, compare_graph_measurements, print_comparison_results
from graph_to_dataframe import graph_to_training_dataframe, preprocess_dataframe
from recreate_graph import get_trained_model, recreate_by_priority_rank

Using TensorFlow backend.


In [2]:
graph_families = [
    {
        'name': 'erdos renyi',
        'func': nx.erdos_renyi_graph,
        'params': [
            {'n': 30, 'p': 0.05},
            {'n': 30, 'p': 0.1},
            {'n': 30, 'p': 0.3}
        ]
    },
    {
        'name': 'watts strogatz',
        'func': nx.watts_strogatz_graph,
        'params': [
            {'n': 30, 'k': 2, 'p': 0.05},
            {'n': 30, 'k': 2, 'p': 0.1},
            {'n': 30, 'k': 2, 'p': 0.3}
        ]
    },
    {
        'name': 'barabasi albert',
        'func': nx.barabasi_albert_graph,
        'params': [
            {'n': 30, 'm': 1},
            {'n': 30, 'm': 2},
            {'n': 30, 'm': 5}
        ]
    },
]

In [3]:
EPOCHS = 64
BATCH_SIZE = 64
VERBOSE = 0

In [4]:
results = defaultdict(list)
for graph_family in graph_families:
    for params in graph_family['params']:
        # Generate graph
        graph = generate_graph(graph_family['func'], params)
        # Set weights to 1 and change graph type to directed
        graph = preprocess_generative_graph(graph)
        # Graph attributes
        attach_graph_attributes(graph)
        # Training df
        df = graph_to_training_dataframe(graph)
        # Preprocess training df
        df = preprocess_dataframe(df, graph.number_of_nodes())
        # Train model
        model = get_trained_model(df, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=VERBOSE)
        
        # drop target column
        X_test = df.drop(['num_of_edges'], axis=1)
        # predict num_edges
        y_pred = model.predict(X_test)
        
        # Recreate graph
        new_graph = recreate_by_priority_rank(graph, y_pred)
        # Get measurements
        graph_measurements = get_graph_measurements(graph)
        new_graph_measurements = get_graph_measurements(new_graph)
        # Make comparison
        comparison = compare_graph_measurements(graph_measurements, new_graph_measurements)
        results[graph_family['name']].append(comparison)

900/900 [==============================] - 0s 50us/step
loss: 0.06522495824119283, accuracy: 0.9355555555555556
Cannot compute average_shortest_path_length - Graph is not weakly connected.
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected


/home/smoq/.virtualenvs/cna/lib/python3.5/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: invalid value encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


900/900 [==============================] - 0s 57us/step
loss: 0.10024502230187257, accuracy: 0.8955555555555555
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected


/home/smoq/.virtualenvs/cna/lib/python3.5/site-packages/networkx/algorithms/assortativity/correlation.py:287: RuntimeWarning: divide by zero encountered in double_scalars
  return (xy*(M-ab)).sum()/numpy.sqrt(vara*varb)


900/900 [==============================] - 0s 52us/step
loss: 0.22213427735699548, accuracy: 0.6844444444444444
900/900 [==============================] - 0s 61us/step
loss: 0.06348890523115794, accuracy: 0.9333333333333333
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 86us/step
loss: 0.06515093313323127, accuracy: 0.9333333333333333
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 82us/step
loss: 0.06695640055462718, accuracy: 0.9333333333333333
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0s 97us/step
loss: 0.06324336966706647, accuracy: 0.9355555555555556
Cannot compute diameter - Found infinite path length because the digraph is not strongly connected
900/900 [==============================] - 0

In [5]:
for key, value in results.items():
    print(key)
    for i, comparison_result in enumerate(value):
        print('graph #{}'.format(i+1))
        print_comparison_results(comparison_result)
        print()

erdos renyi
graph #1
(KS test p-value)  degree_centrality: 0.7600465102607563              [passed: True]
(KS test p-value)  closeness_centrality: 0.025856269697631676         [passed: False]
(KS test p-value)  betweenness_centrality: 0.5371995394073502         [passed: True]
(KS test p-value)  pagerank: 0.011313647722167194                     [passed: False]
(abs distance)     average_shortest_path_length: None                 [passed: None]
(abs distance)     diameter: None                                     [passed: None]
(abs distance)     degree_centralization: 0.3586956521739134          [passed: False]
(abs distance)     closeness_centralization: 0.7018006877894443       [passed: False]
(abs distance)     betweenness_centralization: 0.02513781739218927    [passed: True]
(abs distance)     pagerank_centralization: 0.34434151788137934       [passed: False]
(abs distance)     density: 0.034482758620689655                      [passed: True]
(abs distance)     degree_assortativity